# Attribute

**Original Work**: *Maziar Raissi, Paris Perdikaris, and George Em Karniadakis*

**Github Repo** : https://github.com/maziarraissi/PINNs

**Link:** https://github.com/maziarraissi/PINNs/tree/master/appendix/continuous_time_identification%20(Burgers)

@article{raissi2017physicsI,
  title={Physics Informed Deep Learning (Part I): Data-driven Solutions of Nonlinear Partial Differential Equations},
  author={Raissi, Maziar and Perdikaris, Paris and Karniadakis, George Em},
  journal={arXiv preprint arXiv:1711.10561},
  year={2017}
}

@article{raissi2017physicsII,
  title={Physics Informed Deep Learning (Part II): Data-driven Discovery of Nonlinear Partial Differential Equations},
  author={Raissi, Maziar and Perdikaris, Paris and Karniadakis, George Em},
  journal={arXiv preprint arXiv:1711.10566},
  year={2017}
}

## Libraries and Dependencies

In [15]:
import sys
sys.path.insert(0, '../Utilities/')

import torch
from collections import OrderedDict
from pyDOE import lhs
import numpy as np
# import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
# from plotting import newfig, savefig
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# import matplotlib.gridspec as gridspec
np.random.seed(1234)

In [16]:
# CUDA support
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device = torch.device('cpu')

## Physics-informed Neural Networks

In [17]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()

        # parameters
        self.depth = len(layers) - 1

        # set up layer order dict
        self.activation = torch.nn.Tanh

        layer_list = list()
        for i in range(self.depth - 1):
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))

        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)

        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)

        for module in self.layers.modules():
            if isinstance(module, torch.nn.Linear):
                torch.nn.init.xavier_normal_(module.weight)


    def forward(self, x):
        out = self.layers(x)
        return out

In [18]:
# the physics-guided neural network
class PhysicsInformedNN():
    def __init__(self, X_u, u, X_f, c, all_x, all_t, layers, lb, ub, nu):

        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)

        # data
        self.x_u = torch.tensor(X_u[:, 0:1], requires_grad=True).float().to(device)
        self.t_u = torch.tensor(X_u[:, 1:2], requires_grad=True).float().to(device)
        self.x_f = torch.tensor(X_f[:, 0:1], requires_grad=True).float().to(device)
        self.t_f = torch.tensor(X_f[:, 1:2], requires_grad=True).float().to(device)
        self.all_x = torch.tensor(all_x, requires_grad=True).float().to(device)
        self.all_t = torch.tensor(all_t, requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        self.delta_x = 1/128
        self.c = c

        self.layers = layers
        self.nu = nu

        # deep neural networks
        self.dnn = DNN(layers).to(device)

        # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(),
            lr=1.0,
            max_iter=50000,
            max_eval=50000,
            history_size=50,
            tolerance_grad=1e-5,
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )

        self.iter = 0

    def net_u(self, x, t):

        u = self.dnn(torch.cat([x, t], dim=1))

        volume_x = 2
        delta_x = 1/128

        mesh_t, mesh_x = torch.meshgrid([t.squeeze(1), self.all_x.squeeze(1)], indexing='ij')
        t_by_x = torch.concat((mesh_x.unsqueeze(2), mesh_t.unsqueeze(2)), dim=-1)

        integral_u_dx = torch.sum(self.dnn(t_by_x)*delta_x, dim=1)
        second_term = integral_u_dx / volume_x

        c_tensor = torch.full(x.shape, self.c)
        third_term = c_tensor / volume_x
        return u - second_term + third_term
        return u

    def net_f(self, x, t):
        """ The pytorch autograd version of calculating residual """
        u = self.net_u(x, t)
        u_t = torch.autograd.grad(
            u, t,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_x = torch.autograd.grad(
            u, x,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x,
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]

        f = u_t + u * u_x - self.nu * u_xx
        return f

    def loss_func(self):
        self.optimizer.zero_grad()

        u_pred = self.net_u(self.x_u, self.t_u)
        f_pred = self.net_f(self.x_f, self.t_f)
        loss_u = torch.mean((self.u - u_pred) ** 2)
        loss_f = torch.mean(f_pred ** 2)

        loss = loss_u + loss_f

        loss.backward()

        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_u.item(), loss_f.item())
            )
        return loss

    def train(self):
        self.dnn.train()

        # Backward and optimize
        self.optimizer.step(self.loss_func)


    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t)
        c = torch.sum(u*self.delta_x)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        c = c.detach().cpu().numpy()
        return u, f, c

## Configurations

In [19]:
nu = 0.01/np.pi
noise = 0.0

N_u = 100
N_f = 10000
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

data = scipy.io.loadmat('..data/burgers_shock.mat')

# Pinn paper state data
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T # (100, 256)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]

# Domain bounds
lb = X_star.min(0)
ub = X_star.max(0)

xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]


X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)

c = np.mean(np.sum(Exact*1/128, axis=1))

X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])

idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

1.2856382625159313e-15
[[ 0.04313725  0.        ]
 [ 1.          0.27      ]
 [ 1.          0.11      ]
 [ 1.          0.74      ]
 [-1.          0.79      ]
 [-0.39607843  0.        ]
 [-0.65490196  0.        ]
 [-0.89019608  0.        ]
 [-1.          0.94      ]
 [ 1.          0.88      ]
 [-0.8745098   0.        ]
 [ 0.77254902  0.        ]
 [ 0.50588235  0.        ]
 [ 0.43529412  0.        ]
 [-0.27843137  0.        ]
 [ 1.          0.99      ]
 [ 1.          0.52      ]
 [ 1.          0.83      ]
 [ 1.          0.21      ]
 [ 0.49803922  0.        ]
 [ 0.70196078  0.        ]
 [-1.          0.99      ]
 [-0.09803922  0.        ]
 [ 0.71764706  0.        ]
 [ 1.          0.23      ]
 [-1.          0.61      ]
 [ 1.          0.38      ]
 [-0.21568627  0.        ]
 [-0.15294118  0.        ]
 [-1.          0.48      ]
 [-1.          0.31      ]
 [ 1.          0.13      ]
 [ 1.          0.48      ]
 [ 0.38823529  0.        ]
 [ 0.34117647  0.        ]
 [-1.          0.32      ]
 [ 1.

## Training

In [20]:
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, c, x, t, layers, lb, ub, nu)

In [21]:
%%time

model.train()

Iter 100, Loss: 6.15777e-02, Loss_u: 4.04590e-02, Loss_f: 2.11187e-02
Iter 200, Loss: 1.58252e-02, Loss_u: 9.28122e-03, Loss_f: 6.54397e-03
Iter 300, Loss: 4.67856e-03, Loss_u: 2.28598e-03, Loss_f: 2.39258e-03
Iter 400, Loss: 2.22543e-03, Loss_u: 1.06356e-03, Loss_f: 1.16186e-03
Iter 500, Loss: 1.05869e-03, Loss_u: 3.59107e-04, Loss_f: 6.99582e-04
Iter 600, Loss: 5.35939e-04, Loss_u: 1.36422e-04, Loss_f: 3.99517e-04
Iter 700, Loss: 2.90302e-04, Loss_u: 4.71486e-05, Loss_f: 2.43154e-04
Iter 800, Loss: 1.95125e-04, Loss_u: 3.54913e-05, Loss_f: 1.59634e-04
Iter 900, Loss: 1.50165e-04, Loss_u: 2.60595e-05, Loss_f: 1.24106e-04
Iter 1000, Loss: 1.16857e-04, Loss_u: 2.03339e-05, Loss_f: 9.65233e-05
Iter 1100, Loss: 8.88997e-05, Loss_u: 1.44914e-05, Loss_f: 7.44083e-05
Iter 1200, Loss: 7.40303e-05, Loss_u: 1.27982e-05, Loss_f: 6.12321e-05
Iter 1300, Loss: 6.00087e-05, Loss_u: 9.15713e-06, Loss_f: 5.08515e-05
Iter 1400, Loss: 5.16496e-05, Loss_u: 8.84731e-06, Loss_f: 4.28023e-05
Iter 1500, Loss

In [23]:
u_pred, f_pred, c_pred = model.predict(X_star)

error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)

# c_pred is prediction of c on test set, c_test is groundtruth of c on test set
c_true = np.mean(np.sum(Exact*1/128, axis=1))

error_c = abs(c_pred-c_true)
print('Error u: %e' % (error_u))
print('Error c: %e' % (error_c))

Error u: 4.553463e-03
Error c: 7.546535e-02
